# Импорт данных

Тут импортирую нужные библиотеки и загружаю данные из архива. (Нужно просто загрузить архив на гугл диск и все будет работать)

In [ ]:
import os
import re
import zipfile
from datetime import timedelta

import numpy as np
import pandas as pd
import plotly.express as px
from collections import Counter

!pip install catboost
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score

import nltk
from nltk import pos_tag, word_tokenize

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

from google.colab import drive
drive.mount('/content/drive')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
zip_path = '/content/drive/MyDrive/train_dataset_soc_dem_train.zip'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/train_dataset')
csv_folder = '/content/train_dataset'

csv_files = ['video_info_v2.csv', 'train_targets.csv', 'train_events.csv', 'all_events.csv']

video_info_df = pd.read_csv(f'{csv_folder}/video_info_v2.csv')

train_events_df = pd.read_csv(f'{csv_folder}/train_events.csv')

In [ ]:
# загрузка теста

In [ ]:
zip_path = '/content/drive/MyDrive/test_dataset_Тест.zip'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/test_dataset')
csv_folder = '/content/test_dataset'

test_targets_df = pd.read_csv(f'/content/test_dataset/subm.csv')

test_events_df = pd.read_csv('/content/test_dataset/test_events.csv')

# Feature engineering

Тут достаточно объемная генерация фичей. В итоге получаем датасет final_df, в котором содержится информация о каждом пользователе в тесте.

In [ ]:
events = pd.merge(test_events_df, video_info_df, how = 'left', on = 'rutube_video_id')
timezone_offsets = {
    'Tatarstan Republic': 0,
    'Bashkortostan Republic': 2,
    'Novosibirsk Oblast': 4,
    'Moscow': 0,
    'Moscow Oblast': 0,
    'Omsk Oblast': 3,
    'Chuvashia': 0,
    'Krasnoyarsk Krai': 4,
    'Krasnoyarskiy': 4,
    'Chelyabinsk': 2,
    'Kamchatka': 9,
    'Nizhny Novgorod Oblast': 0,
    'Krasnodar Krai': 0,
    'Volgograd Oblast': 0,
    'Kaliningrad Oblast': -1,
    'Kuzbass': 4,
    'Stavropol Kray': 0,
    'Samara Oblast': 1,
    'Amur Oblast': 6,
    'Sverdlovsk Oblast': 2,
    'St.-Petersburg': 0,
    'Yamalo-Nenets': 2,
    'Orenburg Oblast': 2,
    'Khanty-Mansia': 2,
    'Kaluga Oblast': 0,
    'Tomsk Oblast': 4,
    'Novgorod Oblast': 0,
    'Arkhangelskaya': 0,
    'North Ossetia–Alania': 0,
    'Kursk Oblast': 0,
    "Leningradskaya Oblast'": 0,
    'Ivanovo Oblast': 0,
    'Altay Kray': 4,
    'Kurgan Oblast': 2,
    'Kostroma Oblast': 0,
    'Bryansk Oblast': 0,
    'Dagestan': 0,
    'Lipetsk Oblast': 0,
    'Vladimir Oblast': 0,
    'Kirov Oblast': 0,
    'Khabarovsk': 7,
    'Tambov Oblast': 0,
    'Chukotka': 9,
    'Voronezh Oblast': 0,
    'Sverdlovsk': 2,
    'Tula Oblast': 0,
    'Krasnodarskiy': 0,
    'Irkutsk Oblast': 5,
    'Saratov Oblast': 0,
    'Khakasiya Republic': 4,
    'Penza': 0,
    'Perm Krai': 2,
    'Oryol oblast': 0,
    'Vladimir': 0,
    'Smolensk Oblast': 0,
    'Penza Oblast': 0,
    'Mordoviya Republic': 0,
    'Tyumen’ Oblast': 2,
    'Sakha': 8,
    'Primorye': 7,
    'Zabaykalskiy (Transbaikal) Kray': 6,
    'Vologda Oblast': 0,
    'Yaroslavl Oblast': 0,
    'Crimea': 0,
    'Rostov': 0,
    'Ryazan Oblast': 0,
    'Perm': 2,
    'Chechnya': 0,
    'Udmurtiya Republic': 1,
    'Tver Oblast': 0,
    'Buryatiya Republic': 5,
    'Belgorod Oblast': 0,
    'Kaluga': 0,
    'Astrakhan Oblast': 0,
    'Karelia': 0,
    'Murmansk': 0,
    'Adygeya Republic': 0,
    'Kemerovo Oblast': 4,
    'Mariy-El Republic': 0,
    'Kursk': 0,
    'Saratovskaya Oblast': 0,
    'Sakhalin Oblast': 8,
    'Ivanovo': 0,
    'Tyumen Oblast': 2,
    'Stavropol’ Kray': 0,
    'Voronezj': 0,
    'Karachayevo-Cherkesiya Republic': 0,
    'Kabardino-Balkariya Republic': 0,
    'Ulyanovsk': 1,
    'North Ossetia': 0,
    'Komi': 0,
    'Smolensk': 0,
    'Tver’ Oblast': 0,
    'Sebastopol City': 0,
    'Pskov Oblast': 0,
    'Tula': 0,
    'Orel Oblast': 0,
    'Jaroslavl': 0,
    'Tambov': 0,
    'Kalmykiya Republic': 0,
    'Primorskiy (Maritime) Kray': 7,
    'Altai': 4,
    'Magadan Oblast': 8,
    'Vologda': 0,
    'Tyva Republic': 4,
    'Nenets': 2,
    'Smolenskaya Oblast’': 0,
    'Jewish Autonomous Oblast': 7,
    'Astrakhan': 0,
    'Ingushetiya Republic': 0,
    'Kirov': 0,
    'Transbaikal Territory': 6,
    'Omsk': 3,
    'Kaliningrad': -1,
    'Stavropol Krai': 0,
    'Arkhangelsk Oblast': 0
}

events['event_timestamp'] = pd.to_datetime(events['event_timestamp'])

events['duration'] = pd.to_numeric(events['duration'], errors='coerce')
events['total_watchtime'] = pd.to_numeric(events['total_watchtime'], errors='coerce')

def adjust_timestamp(row):
    offset = timezone_offsets.get(row['region'], 0)
    return row['event_timestamp'] + timedelta(hours=offset)

events['event_timestamp_local'] = events.apply(adjust_timestamp, axis=1)

events['ua_os'].fillna(events['ua_os'].mode()[0], inplace=True)

In [ ]:
test_events_df

,event_timestamp,region,ua_device_type,ua_client_type,ua_os,ua_client_name,total_watchtime,rutube_video_id,viewer_uid
0,2024-06-01 15:26:44+03:00,Udmurtiya Republic,desktop,browser,Windows,Yandex Browser,2593,video_212730,22206
1,2024-06-01 12:25:29+03:00,Adygeya Republic,smartphone,mobile app,Android,Rutube,960,video_235114,34531
2,2024-06-01 17:23:12+03:00,Astrakhan Oblast,smartphone,mobile app,Android,Rutube,4695,video_26520,25830
3,2024-06-01 15:37:37+03:00,Khakasiya Republic,smartphone,browser,Android,Chrome Mobile,2490,video_465561,14838
4,2024-06-01 21:30:11+03:00,Moscow,smartphone,mobile app,Android,Rutube,1117,video_102934,13718
...,...,...,...,...,...,...,...,...,...
587730,2024-06-30 17:33:36+03:00,Krasnodar Krai,smartphone,mobile app,Android,Rutube,2509,video_243869,23170
587731,2024-06-30 12:36:00+03:00,Moscow,tablet,browser,Android,Firefox Mobile,555,video_193920,43241
587732,2024-06-30 06:55:31+03:00,Leningradskaya Oblast',desktop,browser,Windows,Chrome,1713,video_246870,31204
587733,2024-06-30 00:16:31+03:00,Saratov Oblast,smartphone,browser,Android,Chrome Mobile,51,video_15940,40889


In [ ]:
events

,event_timestamp,region,ua_device_type,ua_client_type,ua_os,ua_client_name,total_watchtime,rutube_video_id,viewer_uid,title,category,duration,author_id,event_timestamp_local
0,2024-06-01 15:26:44+03:00,Udmurtiya Republic,desktop,browser,Windows,Yandex Browser,2593,video_212730,22206,Отчаянные Домохозяйки 7 сезон 21 серия,Сериалы,2610785,1089828,2024-06-01 16:26:44+03:00
1,2024-06-01 12:25:29+03:00,Adygeya Republic,smartphone,mobile app,Android,Rutube,960,video_235114,34531,Вот как воевали США во Второй мировой войне!,Разное,1080320,1009406,2024-06-01 12:25:29+03:00
2,2024-06-01 17:23:12+03:00,Astrakhan Oblast,smartphone,mobile app,Android,Rutube,4695,video_26520,25830,"Бесподобный мистер Фокс (мультфильм, 2009)",Фильмы,5208416,1090779,2024-06-01 17:23:12+03:00
3,2024-06-01 15:37:37+03:00,Khakasiya Republic,smartphone,browser,Android,Chrome Mobile,2490,video_465561,14838,2- Вышивальные планы на 2024г,Хобби,2556011,1017105,2024-06-01 19:37:37+03:00
4,2024-06-01 21:30:11+03:00,Moscow,smartphone,mobile app,Android,Rutube,1117,video_102934,13718,"Суперниндзя. Дети, 3 выпуск",Телепередачи,8061640,1009210,2024-06-01 21:30:11+03:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
587730,2024-06-30 17:33:36+03:00,Krasnodar Krai,smartphone,mobile app,Android,Rutube,2509,video_243869,23170,"ТАК СКАЗАТЬ: Фидаров – о феминизме, поствоенно...",Интервью,3524374,1022719,2024-06-30 17:33:36+03:00
587731,2024-06-30 12:36:00+03:00,Moscow,tablet,browser,Android,Firefox Mobile,555,video_193920,43241,Топ-5 Игр На Телефон 2024!,Видеоигры,220550,1018241,2024-06-30 12:36:00+03:00
587732,2024-06-30 06:55:31+03:00,Leningradskaya Oblast',desktop,browser,Windows,Chrome,1713,video_246870,31204,"Творческая встреча с известным писателем, публ...",Интервью,2855611,1010000,2024-06-30 06:55:31+03:00
587733,2024-06-30 00:16:31+03:00,Saratov Oblast,smartphone,browser,Android,Chrome Mobile,51,video_15940,40889,СЕРИАЛЫ 2024 | ПОДБОРКА | SUDARIKOVA KATE,Сериалы,247501,1012086,2024-06-30 00:16:31+03:00


In [ ]:
agg_funcs = {
    'total_watchtime': ['sum', 'mean', 'max', 'median', 'std'],  # Общее, среднее, максимальное время просмотра
    'rutube_video_id': 'count',  # Количество просмотренных видео
    'category': [lambda x: x.mode()[0], lambda x: ' '.join(x)],
    'duration': ['sum', 'mean', 'median', 'std'],  # Общее и среднее время длительности видео
    'title': lambda x: ' '.join(x),  # Сложение заголовков видео
    'region': [lambda x: x.mode()[0], lambda x: x.nunique()],  # Мода по столбцу region
    'ua_os': [lambda x: x.mode()[0], lambda x: x.nunique()],
    'ua_client_name': [lambda x: x.mode()[0], lambda x: x.nunique()],

}

user_history = events.groupby('viewer_uid').agg(agg_funcs)

user_history.columns = ['_'.join(col) if isinstance(col, tuple) else col for col in user_history.columns]

# Подсчет % соотношения ua_device_type
device_counts = events.groupby(['viewer_uid', 'ua_device_type']).size().unstack(fill_value=0)
device_percentage = device_counts.div(device_counts.sum(axis=1), axis=0).fillna(0)
device_percentage.columns = [f'device_percentage_{col}' for col in device_percentage.columns]

# Подсчет % соотношения ua_client_type
client_counts = events.groupby(['viewer_uid', 'ua_client_type']).size().unstack(fill_value=0)
client_percentage = client_counts.div(client_counts.sum(axis=1), axis=0).fillna(0)
client_percentage.columns = [f'client_percentage_{col}' for col in client_percentage.columns]

# Подсчет % видео в каждой категории
category_counts = events.groupby(['viewer_uid', 'category']).size().unstack(fill_value=0)
category_percentage = category_counts.div(category_counts.sum(axis=1), axis=0).fillna(0)
category_percentage.columns = [f'category_percentage_{col}' for col in category_percentage.columns]

total_watchtime_per_user = events.groupby('viewer_uid')['total_watchtime'].sum()
category_watchtime_per_user = events.groupby(['viewer_uid', 'category'])['total_watchtime'].sum()
category_watchtime_ratio = category_watchtime_per_user.div(total_watchtime_per_user, level='viewer_uid').fillna(0)
category_watchtime_ratio = category_watchtime_ratio.unstack(fill_value=0)
category_watchtime_ratio.columns = [f'category_watchtime_ratio_{col}' for col in category_watchtime_ratio.columns]

final_df = pd.merge(user_history, test_targets_df, on='viewer_uid', how='left')
final_df = final_df.join(category_watchtime_ratio, on='viewer_uid')
final_df = final_df.join(device_percentage, on='viewer_uid')
final_df = final_df.join(client_percentage, on='viewer_uid')
final_df = final_df.join(category_percentage, on='viewer_uid')

final_df['mean_len'] = len(final_df['title_<lambda>']) / final_df['rutube_video_id_count']
final_df['mean_upper'] = final_df['title_<lambda>'].apply(lambda x: sum(1 for c in x if c.isupper())) / final_df['rutube_video_id_count']


def top_50_words(titles):
    words = titles.lower().split()
    word_counts = Counter(words)
    most_common_words = word_counts.most_common(10)
    return ' '.join([word for word, _ in most_common_words])

def top_5000_words(titles):
    words = titles.lower().split()
    word_counts = Counter(words)
    most_common_words = word_counts.most_common(1000)
    return ' '.join([word for word, _ in most_common_words])

def top_500_words(titles):
    words = titles.lower().split()
    word_counts = Counter(words)
    most_common_words = word_counts.most_common(100)
    return ' '.join([word for word, _ in most_common_words])


final_df['top_50_frequent_words'] = final_df['title_<lambda>'].apply(top_50_words)
final_df['top_500_frequent_words'] = final_df['title_<lambda>'].apply(top_500_words)
final_df['top_5000_frequent_words'] = final_df['title_<lambda>'].apply(top_5000_words)


# top_100_videos = (train_events_df.groupby('rutube_video_id')['viewer_uid']
#                   .nunique()
#                   .sort_values(ascending=False)
#                   .head(500)
#                   .index)

# df_videos_loaded = pd.read_csv('/content/drive/MyDrive/top_100_videos.csv')
# top_100_videos = df_videos_loaded['rutube_video_id'].tolist()


# top_videos_events = test_events_df[test_events_df['rutube_video_id'].isin(top_100_videos)]


# user_top_videos = (top_videos_events
#                    .assign(watched=1)
#                    .pivot_table(index='viewer_uid',
#                                 columns='rutube_video_id',
#                                 values='watched',
#                                 fill_value=0))

# final_df = final_df.merge(user_top_videos, on='viewer_uid', how='left').fillna(0)

final_df['unique_words'] = final_df['title_<lambda>'].apply(lambda x: ' '.join(set(x.split())))

In [ ]:
final_df

,viewer_uid,total_watchtime_sum,total_watchtime_mean,total_watchtime_max,total_watchtime_median,total_watchtime_std,rutube_video_id_count,category_<lambda_0>,category_<lambda_1>,duration_sum,...,video_94408,video_95009,video_95219,video_95581,video_96465,video_96775,video_97997,video_98905,video_9992,unique_words
0,0,14419,1602.111111,4821,1542.0,1517.068674,9,Интервью,Здоровье Обучение Интервью Интервью Интервью И...,29170703,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"на смерти Goblin не Швейцарии, и управлять в Б..."
1,1,92,92.000000,92,92.0,0.000000,1,Интервью,Интервью,1601680,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Поле / запретят Самоубийство аборты? народа: б...
2,2,3470,1735.000000,2560,1735.0,1166.726189,2,Интервью,Интервью Интервью,7203796,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""ОСД"" КЭПА, ЭЛЬДАРОВ БИЛЛИ ЛЮКА СКАЙУОКЕРА, ДО..."
3,3,4478,1119.500000,1202,1097.0,55.548777,4,Аниме,Аниме Аниме Аниме Аниме,4532333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"бессмертию, серия 108 106 к 107 Путешествие 104"
4,4,73422,1411.961538,4919,1402.5,877.269739,52,Сериалы,Сериалы Сериалы Сериалы Сериалы Сериалы Сериал...,137341970,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Прометей 16 кино 4 (2012) 14 19 Братья смерть ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59999,59999,5131,1026.200000,3213,367.0,1320.525539,5,Телепередачи,Телепередачи Телепередачи Фильмы Телепередачи ...,22128920,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Новая Богиня феминистки ясновидящей Куруч, Гра..."
60000,60000,807,403.500000,757,403.5,499.924494,2,Лайфстайл,Лайфстайл Телепередачи,8327256,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"24 Новая сезон, про скитальцев. экстрасенсов, ..."
60001,60001,1780,1780.000000,1780,1780.0,0.000000,1,Обучение,Обучение,380820,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,БЛОГЕРУ: КАМЕРА? АЙФОН НА ЧТО ИЛИ СНИМАТЬ
60002,60002,5133,2566.500000,4941,2566.5,3358.050104,2,Телепередачи,Телепередачи Телепередачи,11335000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Дубае, Выжить 3 в выпуск 2"


In [ ]:
def extract_additional_text_features(df):
    # Количество уникальных слов в заголовке
    df['title_unique_word_count'] = df['title'].apply(lambda x: len(set(x.split())) if isinstance(x, str) else 0)

    # Наличие специальных символов (например, '!', '?') в заголовке
    df['title_has_special_char'] = df['title'].apply(lambda x: int(bool(re.search(r'[!?]', x))) if isinstance(x, str) else 0)

    # Соотношение заглавных букв к общему количеству букв в заголовке
    df['title_upper_ratio'] = df['title'].apply(lambda x: sum(1 for c in x if c.isupper()) / len(x) if isinstance(x, str) and len(x) > 0 else 0)

    # Первое и последнее слово заголовка
    df['first_word'] = df['title'].apply(lambda x: x.split()[0] if len(x.split()) > 0 else '')
    df['last_word'] = df['title'].apply(lambda x: x.split()[-1] if len(x.split()) > 0 else '')

    # Количество заглавных слов в заголовке
    df['title_upper_word_count'] = df['title'].apply(lambda x: sum(1 for word in x.split() if word.isupper()) if isinstance(x, str) else 0)

    # Средняя длина слов в заголовке
    df['title_avg_word_length'] = df['title'].apply(lambda x: np.mean([len(word) for word in x.split()]) if len(x.split()) > 0 else 0)



    # Наличие иностранных слов (например, английских)
    df['title_foreign_words'] = df['title'].apply(lambda x: int(any(re.search(r'[a-zA-Z]', word) for word in x.split())) if isinstance(x, str) else 0)


    return df

# Применяем функции для извлечения дополнительных текстовых фичей
video_info_df = extract_additional_text_features(video_info_df)

# Объединяем данные с events
events_with_title = pd.merge(events, video_info_df[['rutube_video_id', 'title_unique_word_count', 'title_has_special_char',
                                                    'title_upper_ratio', 'first_word', 'last_word', 'title_upper_word_count',
                                                    'title_avg_word_length',
                                                    'title_foreign_words']],
                             on='rutube_video_id', how='left')

text_agg_funcs = {
    'title_unique_word_count': ['sum', 'mean'],  # Количество уникальных слов
    'title_has_special_char': 'mean',  # Наличие специальных символов
    'title_upper_ratio': 'mean',  # Соотношение заглавных букв
    'title_upper_word_count': ['sum', 'mean'],  # Количество заглавных слов
    'title_avg_word_length': 'mean',  # Средняя длина слов
    'title_foreign_words': 'sum',  # Количество заголовков с иностранными словами
}

user_text_features = events_with_title.groupby('viewer_uid').agg(text_agg_funcs)
user_text_features.columns = ['_'.join(col) if isinstance(col, tuple) else col for col in user_text_features.columns]

final_df = final_df.join(user_text_features, on='viewer_uid')

events['day_of_week'] = events['event_timestamp_local'].dt.day_name()  # Extract day of the week
events['hour'] = events['event_timestamp_local'].dt.hour  # Extract hour of the day
events['time_of_day'] = pd.cut(events['hour'], bins=[0, 6, 12, 18, 24], labels=['night', 'morning', 'day', 'evening'], right=False)

common_time_of_day = events.groupby('viewer_uid')['time_of_day'].agg(lambda x: x.mode()[0])

final_df = final_df.join(common_time_of_day.rename('common_time_of_day'), on='viewer_uid')



In [ ]:
print(final_df.shape)
final_df.head()

(60004, 624)


,viewer_uid,total_watchtime_sum,total_watchtime_mean,total_watchtime_max,total_watchtime_median,total_watchtime_std,rutube_video_id_count,category_<lambda_0>,category_<lambda_1>,duration_sum,...,unique_words,title_unique_word_count_sum,title_unique_word_count_mean,title_has_special_char_mean,title_upper_ratio_mean,title_upper_word_count_sum,title_upper_word_count_mean,title_avg_word_length_mean,title_foreign_words_sum,common_time_of_day
0,0,14419,1602.111111,4821,1542.0,1517.068674,9,Интервью,Здоровье Обучение Интервью Интервью Интервью И...,29170703,...,"на смерти Goblin не Швейцарии, и управлять в Б...",101,11.222222,0.111111,0.089505,8,0.888889,5.784168,2,night
1,1,92,92.000000,92,92.0,0.000000,1,Интервью,Интервью,1601680,...,Поле / запретят Самоубийство аборты? народа: б...,8,8.000000,1.000000,0.037736,0,0.000000,5.750000,0,morning
2,2,3470,1735.000000,2560,1735.0,1166.726189,2,Интервью,Интервью Интервью,7203796,...,"""ОСД"" КЭПА, ЭЛЬДАРОВ БИЛЛИ ЛЮКА СКАЙУОКЕРА, ДО...",30,15.000000,0.000000,0.770000,28,14.000000,5.733333,0,day
3,3,4478,1119.500000,1202,1097.0,55.548777,4,Аниме,Аниме Аниме Аниме Аниме,4532333,...,"бессмертию, серия 108 106 к 107 Путешествие 104",20,5.000000,0.000000,0.028571,0,0.000000,6.200000,0,day
4,4,73422,1411.961538,4919,1402.5,877.269739,52,Сериалы,Сериалы Сериалы Сериалы Сериалы Сериалы Сериал...,137341970,...,Прометей 16 кино 4 (2012) 14 19 Братья смерть ...,410,7.884615,0.000000,0.039572,0,0.000000,5.568338,50,evening


In [ ]:
#final_df.to_csv('/content/drive/MyDrive/final_df_for_learnong3.csv')

# Предикт


In [ ]:
cat_features=['category_<lambda_0>', 'region_<lambda_0>', 'ua_client_name_<lambda_0>', 'ua_os_<lambda_0>', 'common_time_of_day']
              #'cluster_2', 'cluster_3', 'cluster_4', 'cluster_5', 'cluster_6', 'cluster_7', 'cluster_8', 'cluster_9', 'cluster_10']

text_features = ['title_<lambda>', 'top_50_frequent_words','top_500_frequent_words', 'category_<lambda_1>', 'top_5000_frequent_words', 'unique_words']

In [ ]:
model_age_class = CatBoostClassifier(cat_features = cat_features, text_features = text_features, verbose=100, task_type='GPU',
                                     learning_rate=0.051431,
                                     depth=4,
                                     iterations=5262,
                                     l2_leaf_reg=8.737,
                                     )

model_sex = CatBoostClassifier(cat_features = cat_features, text_features = text_features, verbose=100, task_type='GPU',
                                     learning_rate=0.051431,
                                     depth=4,
                                     iterations=5262,
                                     l2_leaf_reg=8.737,
                               )
model_sex.load_model('/content/drive/MyDrive/Copy of model_sex3.pt')

model_age_class.load_model('/content/drive/MyDrive/Copy of model_age_class3.pt')

In [ ]:
y_age_class_pred = model_age_class.predict(final_df.drop(columns = ['age', 'age_class', 'sex']))
y_sex_pred = model_sex.predict(final_df.drop(columns = ['age', 'age_class', 'sex']))

In [ ]:
final_df['age_class'] = y_age_class_pred
final_df['sex'] = y_sex_pred
sample = pd.read_csv('/content/test_dataset/subm.csv')
sample_final = pd.merge(sample.drop(columns = ['age_class', 'sex']), final_df[['viewer_uid', 'age_class', 'sex']], how = 'left', on = 'viewer_uid')
columns = list(sample_final.columns)

age_index = columns.index('age_class')
sex_index = columns.index('sex')

columns[age_index], columns[sex_index] = columns[sex_index], columns[age_index]

index = sample_final[sample_final['age_class'] == 1].index[2]
sample_final.at[index, 'age_class'] = 0

sample_final = sample_final[columns]

,viewer_uid,total_watchtime_sum,total_watchtime_mean,total_watchtime_max,total_watchtime_median,total_watchtime_std,rutube_video_id_count,category_<lambda_0>,category_<lambda_1>,duration_sum,...,unique_words,title_unique_word_count_sum,title_unique_word_count_mean,title_has_special_char_mean,title_upper_ratio_mean,title_upper_word_count_sum,title_upper_word_count_mean,title_avg_word_length_mean,title_foreign_words_sum,common_time_of_day
0,0,14419,1602.111111,4821,1542.0,1517.068674,9,Интервью,Здоровье Обучение Интервью Интервью Интервью И...,29170703,...,"на смерти Goblin не Швейцарии, и управлять в Б...",101,11.222222,0.111111,0.089505,8,0.888889,5.784168,2,night
1,1,92,92.000000,92,92.0,NaN,1,Интервью,Интервью,1601680,...,Поле / запретят Самоубийство аборты? народа: б...,8,8.000000,1.000000,0.037736,0,0.000000,5.750000,0,morning
2,2,3470,1735.000000,2560,1735.0,1166.726189,2,Интервью,Интервью Интервью,7203796,...,"""ОСД"" КЭПА, ЭЛЬДАРОВ БИЛЛИ ЛЮКА СКАЙУОКЕРА, ДО...",30,15.000000,0.000000,0.770000,28,14.000000,5.733333,0,day
3,3,4478,1119.500000,1202,1097.0,55.548777,4,Аниме,Аниме Аниме Аниме Аниме,4532333,...,"бессмертию, серия 108 106 к 107 Путешествие 104",20,5.000000,0.000000,0.028571,0,0.000000,6.200000,0,day
4,4,73422,1411.961538,4919,1402.5,877.269739,52,Сериалы,Сериалы Сериалы Сериалы Сериалы Сериалы Сериал...,137341970,...,Прометей 16 кино 4 (2012) 14 19 Братья смерть ...,410,7.884615,0.000000,0.039572,0,0.000000,5.568338,50,evening
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59999,59999,5131,1026.200000,3213,367.0,1320.525539,5,Телепередачи,Телепередачи Телепередачи Фильмы Телепередачи ...,22128920,...,"Новая Богиня феминистки ясновидящей Куруч, Гра...",43,8.600000,0.000000,0.167177,5,1.000000,5.079304,0,evening
60000,60000,807,403.500000,757,403.5,499.924494,2,Лайфстайл,Лайфстайл Телепередачи,8327256,...,"24 Новая сезон, про скитальцев. экстрасенсов, ...",15,7.500000,0.500000,0.177273,3,1.500000,5.714286,0,evening
60001,60001,1780,1780.000000,1780,1780.0,NaN,1,Обучение,Обучение,380820,...,БЛОГЕРУ: КАМЕРА? АЙФОН НА ЧТО ИЛИ СНИМАТЬ,7,7.000000,1.000000,0.804878,7,7.000000,5.000000,0,evening
60002,60002,5133,2566.500000,4941,2566.5,3358.050104,2,Телепередачи,Телепередачи Телепередачи,11335000,...,"Дубае, Выжить 3 в выпуск 2",10,5.000000,0.000000,0.083333,0,0.000000,4.000000,0,evening


In [ ]:
sample_final.to_csv('sample_c_6.csv', index = False)